<a href="https://colab.research.google.com/github/jo1jun/Human-Action-Recog-VIBE/blob/main/HAMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google Drive Mount

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare VIBE

In [2]:
# Clone the repo
!git clone https://github.com/mkocabas/VIBE.git
%cd VIBE/
# Install the other requirements
!pip install torch==1.4.0 numpy==1.17.5
!pip install git+https://github.com/giacaglia/pytube.git --upgrade
!pip install -r requirements.txt
# Download pretrained weights and SMPL data
!source scripts/prepare_data.sh

Cloning into 'VIBE'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 408 (delta 15), reused 3 (delta 0), pack-reused 373
Receiving objects: 100% (408/408), 15.11 MiB | 45.51 MiB/s, done.
Resolving deltas: 100% (184/184), done.
/content/VIBE
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 20.0 MB 12.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0

  Cloning https://github.com/giacaglia/pytube.git to /tmp/pip-req-build-vx9b7zkw
  Running command git clone -q https://github.com/giacaglia/pytube.git /tmp/pip-req-build-vx9b7zkw
  Created wheel for pytube: filename=pytube-9.5.3-py3-none-any.whl size=32290 sha256=38a1c9fba7d94445963dcf47a4d2324010d1df1108555c6b64bab4d94f5566ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-mrar8r39/wheels/ca/b9/e4/f14415300036c42fae7c12b5dd718f0e8a10e29184fc82eb47
Successfully built pytube
  Cloning https://github.com/mattloper/chumpy.git to /tmp/pip-req-build-m2himypj
  Running command git clone -q https://github.com/mattloper/chumpy.git /tmp/pip-req-build-m2himypj
  Cloning https://github.com/mkocabas/yolov3-pytorch.git to /tmp/pip-req-build-h9y2lb4x
  Running command git clone -q https://github.com/mkocabas/yolov3-pytorch.git /tmp/pip-req-build-h9y2lb4x
  Cloning https://github.com/mkocabas/multi-person-tracker.git to /tmp/pip-req-build-9df33zay
  Running command git clone -q https://github.com/

Downloading...
From: https://drive.google.com/uc?id=1untXhYOLQtpNEy4GTY_0fL_H-k6cTf_r
To: /content/VIBE/data/vibe_data.zip
100% 561M/561M [00:07<00:00, 74.4MB/s]
Archive:  vibe_data.zip
   creating: vibe_data/
  inflating: vibe_data/smpl_mean_params.npz  
  inflating: vibe_data/vibe_model_w_3dpw.pth.tar  
  inflating: vibe_data/gmm_08.pkl    
  inflating: vibe_data/J_regressor_h36m.npy  
  inflating: vibe_data/vibe_model_wo_3dpw.pth.tar  
  inflating: vibe_data/SMPL_NEUTRAL.pkl  
  inflating: vibe_data/J_regressor_extra.npy  
  inflating: vibe_data/spin_model_checkpoint.pth.tar  
  inflating: vibe_data/sample_video.mp4  
  inflating: vibe_data/yolov3.weights  


## **Caution**

device & torch version must be cuda & 1.4.0 respectively

If torch version is not 1.4.0, restart runtime (ctrl + M) and run from below cell.

In [3]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
print(torch.__version__)

%cd /content/VIBE

cuda
1.4.0
/content/VIBE


# Custom Dataset & DataLoader

In [4]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
# from PIL import Image
import cv2
import json

# custom dataset 만들기.
class UcfDataset(Dataset):
    def __init__(self, base_dir):
 
        # 경로 설정.
        with open('/content/drive/Shareddrives/human_pose_classification/dataset/actions/walk_run.json') as json_file:
            json_data = json.load(json_file)

        self.img_names = []
        self.annotations = []
        self.actions = []
        # action_type 0 -> Run-Side, 1 -> Walk-Front
        action_type = -1 
        for action in list(json_data.keys()):
            action_dir = os.path.join(base_dir, action)
            action_type += 1
            for video_num in list(json_data[action].keys()):
                # 아직 완성되지 않은 video는 예외처리.
                if len(json_data[action][video_num]) == 0:
                    continue

                cur_dir = os.path.join(action_dir, video_num)

                img_list = json_data[action][video_num]['images']
                self.img_names.append([os.path.join(cur_dir, f) for f in img_list])

                self.annotations.append(json_data[action][video_num]['bboxes'])

                self.actions.append(action_type)

    def __len__(self):
        return len(self.img_names) # 전체 비디오 개수

    def __getitem__(self, index):
        return self.img_names[index], self.annotations[index], self.actions[index] # 전체 비디오중 특정 하나를 indexing 하여 불러옴.


In [5]:
# sampling & cropping & normalizing code
from google.colab.patches import cv2_imshow
from lib.data_utils.img_utils import get_single_image_crop_demo
import random
from torch.utils.data import DataLoader
import cv2

dataset = UcfDataset("/content/drive/Shareddrives/human_pose_classification/dataset/actions/")

bbox_scale = 1.1
crop_size = 224

def make_sampling(data):

    # data : [batch_size, 3] # 3 : image, bbox, action 을 의미.

    batch_size = len(data)
    seq_len = 16

    batch_images  = []
    batch_actions = []

    for i in range(batch_size):
      images = data[i][0]
      bboxes = data[i][1]
      action = data[i][2]

      # images : [frame_len, height, width, channel]
      # bboxes : [frame_len, 4]
      # action : [1]

      sample_images = []
      sample_bboxes = []
      idx = random.randrange(0, len(images) - seq_len)

      for _ in range(seq_len):
          sample_images.append(cv2.imread(images[idx])[:,:,::-1].copy().astype(np.float32))
          sample_bboxes.append(bboxes[idx])
          idx = idx + 1

      # sample_images : [seq_len, height, width, channel]
      # sample_bboxes : [seq_len, 4]
      # action        : [1]

      norm_imgs = []
    
      for img, bbox in zip(sample_images, sample_bboxes):

        norm_img, raw_img, kp_2d = get_single_image_crop_demo(
            img,
            bbox,
            None,
            bbox_scale,
            crop_size)
        norm_imgs.append(norm_img.unsqueeze(0))

      norm_sample_images = torch.cat(norm_imgs)
      # norm_sample_images : [seq_len, 3, 224, 224]
      batch_images.append(norm_sample_images.unsqueeze(0))
      batch_actions.append(action)

    batch_images = torch.cat(batch_images)
    batch_actions = torch.Tensor(batch_actions)

    return batch_images, batch_actions

# # test code
# dataloader = DataLoader(dataset, shuffle=True, collate_fn=make_sampling, batch_size=4)
# for i, (batch_images, batch_actions) in enumerate(dataloader):
#   print(batch_images.shape)
#   print(batch_actions.shape)


# Pretrained VIBE model

In [6]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

import cv2
import time
import torch
import joblib
import shutil
import colorsys
import argparse
import numpy as np
import json
from tqdm import tqdm
from torch.utils.data import DataLoader

from lib.models.vibe import VIBE_Demo
from lib.utils.renderer import Renderer
from lib.dataset.inference import Inference
from lib.utils.smooth_pose import smooth_pose
from lib.data_utils.kp_utils import convert_kps
from lib.utils.pose_tracker import run_posetracker

from lib.utils.demo_utils import (
    download_youtube_clip,
    smplify_runner,
    convert_crop_coords_to_orig_img,
    convert_crop_cam_to_orig_img,
    prepare_rendering_results,
    video_to_images,
    images_to_video,
    download_ckpt,
)

MIN_NUM_FRAMES = 16

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# ========= Define VIBE model ========= #
model = VIBE_Demo(seqlen=16, n_layers=2, hidden_size=1024,
    add_linear=True,
    use_residual=True)

model = model.to(device)

# ========= Load pretrained weights ========= #
pretrained_file = download_ckpt(use_3dpw=False)
ckpt = torch.load(pretrained_file)
print(f'Performance of pretrained model on 3DPW: {ckpt["performance"]}')
ckpt = ckpt['gen_state_dict']
model.load_state_dict(ckpt, strict=False)
model.eval()
print(f'Loaded pretrained weights from \"{pretrained_file}\"')

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth



=> loaded pretrained model from 'data/vibe_data/spin_model_checkpoint.pth.tar'
Performance of pretrained model on 3DPW: 56.56075477600098
Loaded pretrained weights from "data/vibe_data/vibe_model_wo_3dpw.pth.tar"


# Classifier (Only MLP)

In [8]:
import torch.nn as nn

class Classifier(nn.Module):
  def __init__(self, seqlen = 16):
    super().__init__()
    self.fc1 = nn.Linear(seqlen * 72 + seqlen * 10, 1024)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(1024, 512)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(512, 256)
    self.relu3 = nn.ReLU()
    self.fc4 = nn.Linear(256, 3)

  def forward(self, parameters):
    x = self.relu1(self.fc1(parameters))
    x = self.relu2(self.fc2(x))
    x = self.relu3(self.fc3(x))
    output = self.fc4(x)
    return output

In [9]:
classifier = Classifier()

# Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()

# Hyperparameters

In [12]:
import torch.optim as optim

learning_rate = 0.00001
batch_size = 30
num_epochs = 10000

optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)

dataloader = DataLoader(dataset, shuffle=True, collate_fn=make_sampling, batch_size=batch_size, drop_last=True) # 마지막 batch drop
# batch 가 영상 개수 기준으로 형성된다.

# Trainer

In [ ]:
classifier = classifier.to(device)
classifier.train()

for e in range(num_epochs):
  for i, (images, action) in enumerate(dataloader):

    optimizer.zero_grad()

    with torch.no_grad():
      
      batch_size = images.shape[0]
      images = images.to(device)
      # images : [batch_size, seqlen, 3, 224, 224]

      output = model(images)[-1]

      # classifier 의 input 으로 활용될 VIBE outputs (parameters)
      poses = output['theta'][:, :, 3:75]
      betas = output['theta'][:, :, 75:]
      joints3d = output['kp_3d']
      joints2d = output['kp_2d']

      # poses : [batch_size, seq_len, 72]
      # betas.shape : [batch_size, seq_len, 10]
      # joints3d : [batch_size, seq_len, 49, 3]
      # joints2d : [batch_size, seq_len, 49, 2]

      # # 첫 3개의 image sequence 의 첫번째 image 를 시각화. batch 안의 action 이 mix 되어있음.
      # cv2_imshow(np.array(images[0,0].permute(1,2,0).cpu()))
      # print(action[0])
      # cv2_imshow(np.array(images[1,0].permute(1,2,0).cpu()))
      # print(action[1])
      # cv2_imshow(np.array(images[2,0].permute(1,2,0).cpu()))
      # print(action[2])

    parameters = torch.cat([poses.reshape(batch_size, -1), betas.reshape(batch_size, -1)], dim=1)
    # parameters : [batch_size, 1312]

    output = classifier(parameters)
    action = action.to(device, torch.long)

    loss = criterion(output, action)

    print(loss)

    loss.backward()
    optimizer.step()

  if e % 100 == 0:
    torch.save(classifier, '/content/drive/Shareddrives/human_pose_classification/model.pt')

tensor(1.2495, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2280, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.1499, device='cuda:0', grad_fn=<NllLossBackward>)


/usr/local/lib/python3.7/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
tensor(0.3230, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4145, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2354, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2486, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4438, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2772, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3555, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2449, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2917, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4731, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3969, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1744, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3208, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4335, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2630, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1980, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4161, devic

# model load

In [10]:
classifier = torch.load('/content/drive/Shareddrives/human_pose_classification/model.pt')

# overfitting check

In [25]:
classifier = classifier.to(device)
classifier.eval()

num_epochs = 100

total_correct_ratio = []
for e in range(num_epochs):
  for i, (images, action) in enumerate(dataloader):

    optimizer.zero_grad()

    with torch.no_grad():
      
      batch_size = images.shape[0]
      images = images.to(device)
      # images : [batch_size, seqlen, 3, 224, 224]

      output = model(images)[-1]

      # classifier 의 input 으로 활용될 VIBE outputs (parameters)
      poses = output['theta'][:, :, 3:75]
      betas = output['theta'][:, :, 75:]
      joints3d = output['kp_3d']
      joints2d = output['kp_2d']

      # poses : [batch_size, seq_len, 72]
      # betas.shape : [batch_size, seq_len, 10]
      # joints3d : [batch_size, seq_len, 49, 3]
      # joints2d : [batch_size, seq_len, 49, 2]

      parameters = torch.cat([poses.reshape(batch_size, -1), betas.reshape(batch_size, -1)], dim=1)
      # parameters : [batch_size, 1312]

      output = classifier(parameters)
    
    predict = np.array(np.argmax(output.cpu(), 1))
    action = np.array(action)
    correct_ratio = (np.sum(action == predict)) / batch_size
    print('correct_ratio : ', correct_ratio)
    total_correct_ratio.append(correct_ratio)

print('total_correct_ratio : ', np.mean(total_correct_ratio))

correct_ratio :  0.9666666666666667
correct_ratio :  0.9333333333333333
correct_ratio :  0.8333333333333334
correct_ratio :  0.8
correct_ratio :  0.9
correct_ratio :  0.9
correct_ratio :  0.9333333333333333
correct_ratio :  0.8666666666666667
correct_ratio :  0.9
correct_ratio :  0.9
correct_ratio :  0.9333333333333333
correct_ratio :  0.8333333333333334
correct_ratio :  0.9333333333333333
correct_ratio :  0.9
correct_ratio :  0.8333333333333334
correct_ratio :  0.9333333333333333
correct_ratio :  0.9
correct_ratio :  0.9333333333333333
correct_ratio :  0.9666666666666667
correct_ratio :  0.8333333333333334
correct_ratio :  0.8
correct_ratio :  0.9666666666666667
correct_ratio :  0.8666666666666667
correct_ratio :  0.8333333333333334
correct_ratio :  0.8666666666666667
correct_ratio :  0.9666666666666667
correct_ratio :  0.9333333333333333
correct_ratio :  0.9333333333333333
correct_ratio :  0.8333333333333334
correct_ratio :  0.8666666666666667
correct_ratio :  0.9333333333333333
corr